In [45]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [32]:
WINDOW_SIZE = 10
NUMBER_OF_FEATURES = 4
NUMBER_OF_CLASSES = 3
DATA_FOLDER = 'C:\Dev\\vr-kat-project-python-2\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

ALL_X_TRAIN = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y training data

In [34]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

# TEMP LIMITER REMOVE WHEN READY
#file_names = [file_names[20]]

print(file_names)

['PROC-TRAIN2-STAND1.xlsx', 'PROC-TRAIN2-STAND2.xlsx', 'PROC-TRAIN2-STAND3.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-50BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-60BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-80BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-S

In [22]:
def normalizeSensorData(sensor_input):
    if(sensor_input < 0):
        return np.abs(sensor_input/180)
    else:
        return 0.5 + np.abs(sensor_input/180)
    

def df2Xy(df, windowSize=5):
  X, y = [], []


  for i in range( len(df) - windowSize ):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()
        # add row values to the input
        input.append(row_values)

    # add our input to our total inputs, marked as X
    X.append(input)


    # outputs: y labels
    label = df.loc[i + windowSize - 1, ['X_Vel_Positive', 'X_Vel_Negative', 'Z_Vel']].values.tolist()

    y.append(label)


  return (np.array(X), np.array(y))

In [35]:
for fileName in file_names:
    # Read the Excel file
    df = pd.read_excel(DATA_FOLDER + fileName)

    # NORMALIZE THE SENSOR DATA
    df["L_Pitch"] = df["L_Pitch"].apply(normalizeSensorData)
    df["L_Roll"] = df["L_Roll"].apply(normalizeSensorData)
    df["R_Pitch"] = df["R_Pitch"].apply(normalizeSensorData)
    df["R_Roll"] = df["R_Roll"].apply(normalizeSensorData)

    # CREATE NEW COLUMNS FOR Z VELOCITY (POSITIVE AND NEGATIVE)
    df['X_Vel_Positive'] = df['X_Vel'].mask(df['X_Vel'] < 0, 0)
    df['X_Vel_Negative'] = -1*df['X_Vel'].mask(df['X_Vel'] >= 0, 0)


    xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

    ALL_X_TRAIN = np.concatenate([ALL_X_TRAIN, xTrain])
    ALL_Y_TRAIN = np.concatenate([ALL_Y_TRAIN, yTrain])

    
    print('xTrain.shape:', xTrain.shape, ' yTrain.shape:', yTrain.shape)
    print('AllxTrain.shape:', ALL_X_TRAIN.shape, ' AllyTrain.shape:', ALL_Y_TRAIN.shape)
    print('-------------------------------')


xTrain.shape: (1349, 10, 4)  yTrain.shape: (1349, 3)
AllxTrain.shape: (5923, 10, 4)  AllyTrain.shape: (5923, 3)
-------------------------------
xTrain.shape: (3302, 10, 4)  yTrain.shape: (3302, 3)
AllxTrain.shape: (9225, 10, 4)  AllyTrain.shape: (9225, 3)
-------------------------------
xTrain.shape: (1757, 10, 4)  yTrain.shape: (1757, 3)
AllxTrain.shape: (10982, 10, 4)  AllyTrain.shape: (10982, 3)
-------------------------------
xTrain.shape: (3395, 10, 4)  yTrain.shape: (3395, 3)
AllxTrain.shape: (14377, 10, 4)  AllyTrain.shape: (14377, 3)
-------------------------------
xTrain.shape: (3288, 10, 4)  yTrain.shape: (3288, 3)
AllxTrain.shape: (17665, 10, 4)  AllyTrain.shape: (17665, 3)
-------------------------------
xTrain.shape: (3787, 10, 4)  yTrain.shape: (3787, 3)
AllxTrain.shape: (21452, 10, 4)  AllyTrain.shape: (21452, 3)
-------------------------------
xTrain.shape: (3086, 10, 4)  yTrain.shape: (3086, 3)
AllxTrain.shape: (24538, 10, 4)  AllyTrain.shape: (24538, 3)
--------------

In [43]:
model2 = Sequential()
model2.add(InputLayer((10, 4)))
model2.add(LSTM(128))
model2.add(Dense(16, 'relu'))
model2.add(Dense(8, 'relu'))
model2.add(Dense(3, 'relu'))

model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128)               68096     
                                                                 
 dense_7 (Dense)             (None, 16)                2064      
                                                                 
 dense_8 (Dense)             (None, 8)                 136       
                                                                 
 dense_9 (Dense)             (None, 3)                 27        
                                                                 
Total params: 70,323
Trainable params: 70,323
Non-trainable params: 0
_________________________________________________________________


In [47]:
cp2 = ModelCheckpoint('model2/', save_best_only=True)
model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=['acc']) #metrics=[RootMeanSquaredError()])

In [48]:
history2 = model2.fit(ALL_X_TRAIN, ALL_Y_TRAIN, epochs=50, callbacks=[cp2])

Epoch 1/50
4052/4052 [==============================] - 35s 8ms/step - loss: 0.1995 - acc: 0.9480
Epoch 2/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1902 - acc: 0.9583
Epoch 3/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1890 - acc: 0.9625
Epoch 4/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1885 - acc: 0.9625
Epoch 5/50
4052/4052 [==============================] - 38s 9ms/step - loss: 0.1881 - acc: 0.9628
Epoch 6/50
4052/4052 [==============================] - 34s 8ms/step - loss: 0.1874 - acc: 0.9637
Epoch 7/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1845 - acc: 0.9615
Epoch 8/50
4052/4052 [==============================] - 38s 9ms/step - loss: 0.1788 - acc: 0.9629
Epoch 9/50
4052/4052 [==============================] - 41s 10ms/step - loss: 0.1767 - acc: 0.9624
Epoch 10/50
4052/4052 [==============================] - 40s 10ms/step - loss: 0.1750 - acc: 0.9626
Epoch 11/50
4052/